In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys
sys.path.append('../src')

from math_utils import *
from model import *

In [74]:
# Generate random dataset
np.random.seed(42)
x_train = np.random.randn(1000, 3)

# Invent a label based on the three input features
y_train = x_train[:, 0] * 2 + x_train[:, 1] * -1.5 + x_train[:, 2] * 0.5

In [75]:
x_test = np.random.randn(2000, 3)
y_test = x_test[:, 0] * 2 + x_test[:, 1] * -1.5 + x_test[:, 2] * 0.5

In [76]:
model = ELM(3, 200, init_method='normal')

model.fit(x_train, y_train.reshape(-1, 1))

In [77]:
# compute the prediction loss
y_pred = model.predict(x_test)
mse = np.mean((y_test - y_pred) ** 2)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.015241875727959233


In [78]:
# Calculate mean absolute error without sklearn
y_pred_test = model.predict(x_test)
mae = np.mean(np.abs(y_test - y_pred_test))
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.047174655912994974
